## import

In [ ]:
pip install name_matching

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 2.5 MB/s eta 0:00:00


In [ ]:
pip install rapidfuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 28.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

from dateutil import parser
import datetime
from datetime import datetime
from datetime import time
import matplotlib.pyplot as plt
import unicodedata
import re
from tqdm import tqdm
from cleanco import basename

In [ ]:
from rapidfuzz import fuzz, utils
from rapidfuzz import process, fuzz

# Data

In [ ]:
# Preqin
Preqin_deals = pd.read_excel("/content/drive/MyDrive/UT_research/VC_IB/data/preqin/Preqin_deals_1983_2023.xlsx")
Preqin_deals

In [ ]:
sdc_ipo_20230930 = pd.read_csv('/content/drive/MyDrive/UT_research/VC_IB/results/sdc_ipo_20230930.csv')
sdc_ipo_20230930.drop('Unnamed: 0',axis=1,inplace=True)
sdc_ipo_20230930

In [ ]:
sdc_ipo_20230930.columns

In [ ]:
sdc_ipo_20230930['deal_number'].drop_duplicates()

In [ ]:
sdc_ipo_20230930['issuer'].value_counts().head(20)

In [ ]:
# drop at least one is missing
drop_list = ['PORTFOLIO COMPANY ID', 'INVESTORS / BUYERS (FIRMS)','DEAL DATE','PORTFOLIO COMPANY']
Preqin_deals = Preqin_deals.dropna(subset=drop_list)
Preqin_deals["jkl_index2"] = Preqin_deals.index
Preqin_deals

# Rapid fuzz

###  data


In [ ]:
sdc_name_df = pd.DataFrame(sdc_ipo_20230930['issuer']).copy()
sdc_name_df

,issuer
0,DiaSys Corp
1,SunPharm Corp
2,California Pro Sports Inc
3,Infosafe Systems Inc
4,Timeline Inc
...,...
5994,Acrivon Therapeutics Inc
5995,Adamas One Corp
5996,Adamas One Corp
5997,Lipella Pharmaceuticals Inc


In [ ]:
preq_name_df = pd.DataFrame(Preqin_deals['PORTFOLIO COMPANY'])
preq_name_df = preq_name_df.astype(str)
preq_name_df

,PORTFOLIO COMPANY
0,"Maxim Integrated Products, Inc."
1,Bridge Communications Inc.
2,Bridge Communications Inc.
3,Bridge Communications Inc.
4,Bridge Communications Inc.
...,...
738450,Pencil Biosciences Limited
738451,Pencil Biosciences Limited
738452,Hydrexia Pty Ltd
738453,"Suzhou Fumailei Medical Technology Co., Ltd."


In [ ]:
preq_name_df['PORTFOLIO COMPANY'] = [ re.sub(r"[^0-9a-zA-Z\s]", "", pcomp).replace('\t',' ') for pcomp in preq_name_df['PORTFOLIO COMPANY'] ]
# special issue
# preq_name_df.iloc[95985] = "Virtual Monitor_Systems Sp z oo"
# preq_name_df = preq_name_df.drop([95985]).reset_index(drop=True)
preq_name_df_2 = preq_name_df.drop_duplicates(subset=['PORTFOLIO COMPANY']).reset_index().copy()
preq_name_df_2 = preq_name_df_2.rename(columns={'index': "jkl_index2"})
preq_name_df_2

,jkl_index2,PORTFOLIO COMPANY
0,0,Maxim Integrated Products Inc
1,1,Bridge Communications Inc
2,8,Mentor Graphics Corporation
3,12,Oracle Corporation
4,13,Cypress Semiconductor Corporation
...,...,...
129589,738397,Yaysay Inc
129590,738417,BILRS Inc
129591,738423,Breezit Inc
129592,738439,Kamino Retail


### preproccess

In [ ]:
suffix = ["holding",'& Co','Inc','LLC','Corp','Group','Ltd','LP','& co','inc','llc',
          'corp','group','ltd','lp','securities','sec','investment','markets','mkts','usa','capital']
suffix_2 = [' co',' inc',' llc',' corp',' ltd',' lp']
# mannually preproccess the company names
def preprocceser(text_1):
  # removing all capital letters.v
  text_1 = text_1.lower()
  # replace non-ASCII characters.
  text_1 = unicodedata.normalize('NFKD', text_1 ).encode('ASCII', 'ignore').decode()
  # remove any character that is not a word or space character with nothing
  text_1 = re.sub(r'[^\w\s]','',text_1)
  # remove common legal business suffixes and retun
  for word in suffix_2:
        text_1 = re.sub(fr"\b{word}\b", '', text_1)
  return basename(text_1)

In [ ]:
# preproccess first
sdc_name_df['issuer_2'] = sdc_name_df['issuer'].apply(preprocceser)
preq_name_df_2['PORTFOLIO_COMPANY_2'] = preq_name_df_2['PORTFOLIO COMPANY'].apply(preprocceser)

In [ ]:
tempt_list = []

for comp in tqdm(sdc_name_df['issuer_2']):

  tempt_list.append(process.extractOne(comp, preq_name_df_2['PORTFOLIO_COMPANY_2'], scorer=fuzz.WRatio))

100%|██████████| 5999/5999 [14:13<00:00,  7.02it/s]


In [ ]:
matched_df = pd.DataFrame(tempt_list)
# matched_df = matched_df.sort_values(by=1,ascending=False)
matched_df = matched_df.rename(columns={0: "proccessed_preqin_name", 1:"score", 2:'preqin_location'})
matched_df

,proccessed_preqin_name,score,preqin_location
0,diva,77.142857,16532
1,sunpharm,100.000000,500
2,port,90.000000,39423
3,systems,90.000000,782
4,timelines,94.117647,13879
...,...,...,...
5994,acrivon therapeutics,100.000000,109717
5995,one,90.000000,24542
5996,one,90.000000,24542
5997,lipella pharmaceuticals,100.000000,11009


In [ ]:
sdc_name_df['jkl_index'] = sdc_name_df.index
sdc_name_df

,issuer,issuer_2,jkl_index
0,DiaSys Corp,diasys,0
1,SunPharm Corp,sunpharm,1
2,California Pro Sports Inc,california pro sports,2
3,Infosafe Systems Inc,infosafe systems,3
4,Timeline Inc,timeline,4
...,...,...,...
5994,Acrivon Therapeutics Inc,acrivon therapeutics,5994
5995,Adamas One Corp,adamas one,5995
5996,Adamas One Corp,adamas one,5996
5997,Lipella Pharmaceuticals Inc,lipella pharmaceuticals,5997


In [ ]:
matched_df_2 =  pd.concat([sdc_name_df,matched_df],axis=1)
matched_df_2

,issuer,issuer_2,jkl_index,proccessed_preqin_name,score,preqin_location
0,DiaSys Corp,diasys,0,diva,77.142857,16532
1,SunPharm Corp,sunpharm,1,sunpharm,100.000000,500
2,California Pro Sports Inc,california pro sports,2,port,90.000000,39423
3,Infosafe Systems Inc,infosafe systems,3,systems,90.000000,782
4,Timeline Inc,timeline,4,timelines,94.117647,13879
...,...,...,...,...,...,...
5994,Acrivon Therapeutics Inc,acrivon therapeutics,5994,acrivon therapeutics,100.000000,109717
5995,Adamas One Corp,adamas one,5995,one,90.000000,24542
5996,Adamas One Corp,adamas one,5996,one,90.000000,24542
5997,Lipella Pharmaceuticals Inc,lipella pharmaceuticals,5997,lipella pharmaceuticals,100.000000,11009


In [ ]:
preq_name_df_2['preqin_location'] = preq_name_df_2.index
preq_name_df_2

,jkl_index2,PORTFOLIO COMPANY,PORTFOLIO_COMPANY_2,preqin_location
0,0,Maxim Integrated Products Inc,maxim integrated products,0
1,1,Bridge Communications Inc,bridge communications,1
2,8,Mentor Graphics Corporation,mentor graphics,2
3,12,Oracle Corporation,oracle,3
4,13,Cypress Semiconductor Corporation,cypress semiconductor,4
...,...,...,...,...
129589,738397,Yaysay Inc,yaysay,129589
129590,738417,BILRS Inc,bilrs,129590
129591,738423,Breezit Inc,breezit,129591
129592,738439,Kamino Retail,kamino retail,129592


In [ ]:
matched_df_3 =  pd.merge(matched_df_2, preq_name_df_2, on = "preqin_location", how = "left")
matched_df_3 = matched_df_3.rename(columns={'issuer': "sdc_name", 'issuer_2':"proccessed_sdc_name", 'PORTFOLIO COMPANY':'preqin_name'})
matched_df_3

,sdc_name,proccessed_sdc_name,jkl_index,proccessed_preqin_name,score,preqin_location,jkl_index2,preqin_name,PORTFOLIO_COMPANY_2
0,DiaSys Corp,diasys,0,diva,77.142857,16532,75809,Diva AG,diva
1,SunPharm Corp,sunpharm,1,sunpharm,100.000000,500,1089,SunPharm Corp,sunpharm
2,California Pro Sports Inc,california pro sports,2,port,90.000000,39423,175241,PORT Inc,port
3,Infosafe Systems Inc,infosafe systems,3,systems,90.000000,782,1866,SYSTEMS AG,systems
4,Timeline Inc,timeline,4,timelines,94.117647,13879,63397,Timelines,timelines
...,...,...,...,...,...,...,...,...,...
5994,Acrivon Therapeutics Inc,acrivon therapeutics,5994,acrivon therapeutics,100.000000,109717,577557,Acrivon Therapeutics Inc,acrivon therapeutics
5995,Adamas One Corp,adamas one,5995,one,90.000000,24542,112256,One Inc,one
5996,Adamas One Corp,adamas one,5996,one,90.000000,24542,112256,One Inc,one
5997,Lipella Pharmaceuticals Inc,lipella pharmaceuticals,5997,lipella pharmaceuticals,100.000000,11009,50226,Lipella Pharmaceuticals Inc,lipella pharmaceuticals


In [ ]:
matched_df_4 = matched_df_3.loc[:,['jkl_index','jkl_index2','sdc_name','preqin_name','score','proccessed_sdc_name','proccessed_preqin_name']]
matched_df_4

,jkl_index,jkl_index2,sdc_name,preqin_name,score,proccessed_sdc_name,proccessed_preqin_name
0,0,75809,DiaSys Corp,Diva AG,77.142857,diasys,diva
1,1,1089,SunPharm Corp,SunPharm Corp,100.000000,sunpharm,sunpharm
2,2,175241,California Pro Sports Inc,PORT Inc,90.000000,california pro sports,port
3,3,1866,Infosafe Systems Inc,SYSTEMS AG,90.000000,infosafe systems,systems
4,4,63397,Timeline Inc,Timelines,94.117647,timeline,timelines
...,...,...,...,...,...,...,...
5994,5994,577557,Acrivon Therapeutics Inc,Acrivon Therapeutics Inc,100.000000,acrivon therapeutics,acrivon therapeutics
5995,5995,112256,Adamas One Corp,One Inc,90.000000,adamas one,one
5996,5996,112256,Adamas One Corp,One Inc,90.000000,adamas one,one
5997,5997,50226,Lipella Pharmaceuticals Inc,Lipella Pharmaceuticals Inc,100.000000,lipella pharmaceuticals,lipella pharmaceuticals


In [ ]:
matched_df_4= matched_df_4.sort_values(by='score',ascending=False)
matched_df_4

,jkl_index,jkl_index2,sdc_name,preqin_name,score,proccessed_sdc_name,proccessed_preqin_name
2999,2999,943,Sequoia Software Corp,Sequoia Software,100.0,sequoia software,sequoia software
3944,3944,3983,Shutterfly Inc,Shutterfly LLC,100.0,shutterfly,shutterfly
3947,3947,4924,eHealth Inc,eHealth Inc,100.0,ehealth,ehealth
3957,3957,6996,Achillion Pharmaceuticals Inc,Achillion Pharmaceuticals Inc,100.0,achillion pharmaceuticals,achillion pharmaceuticals
3962,3962,39408,ORBCOMM Inc,ORBCOMM Inc,100.0,orbcomm,orbcomm
...,...,...,...,...,...,...,...
3750,3750,811,ev3 Inc,Evolve Software,72.0,ev3,evolve software
1757,1757,78,N2K Inc,Optilink,72.0,n2k,optilink
3926,3926,8420,PGT Inc,Targacept,72.0,pgt,targacept
4831,4831,27083,VWR Corp,Handshake VR Inc,72.0,vwr,handshake vr


In [ ]:
matched_df_4[matched_df_4['score']>97.5]

,jkl_index,jkl_index2,sdc_name,preqin_name,score,proccessed_sdc_name,proccessed_preqin_name
2999,2999,943,Sequoia Software Corp,Sequoia Software,100.000000,sequoia software,sequoia software
3944,3944,3983,Shutterfly Inc,Shutterfly LLC,100.000000,shutterfly,shutterfly
3947,3947,4924,eHealth Inc,eHealth Inc,100.000000,ehealth,ehealth
3957,3957,6996,Achillion Pharmaceuticals Inc,Achillion Pharmaceuticals Inc,100.000000,achillion pharmaceuticals,achillion pharmaceuticals
3962,3962,39408,ORBCOMM Inc,ORBCOMM Inc,100.000000,orbcomm,orbcomm
...,...,...,...,...,...,...,...
771,771,953,Sapient Corp,Sapient,100.000000,sapient,sapient
2524,2524,1603,Aironet Wireless Communication,Aironet Wireless Communications,98.360656,aironet wireless communication,aironet wireless communications
1395,1395,705,Coulter Pharmaceuticals Inc,Coulter Pharmaceutical Inc,97.777778,coulter pharmaceuticals,coulter pharmaceutical
4016,4016,49255,US Auto Parts Networks Inc,US Auto Parts Network Inc,97.674419,us auto parts networks,us auto parts network
